# Tools for a Customer Outreach Campaign

In this lesson, you will learn more about Tools. You'll focus on three key elements of Tools:
- Versatility
- Fault Tolerance
- Caching

- Import libraries, APIs and LLM
- [Serper](https://serper.dev)

In [1]:
# Warning control
import warnings
from crewai import Agent, Task, Crew
warnings.filterwarnings('ignore')

/Users/akaoui/Desktop/Github/ml-projects/.venv/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:898: UserWarning: Mixing V1 models and V2 models (or constructs, like `TypeAdapter`) is not supported. Please upgrade `CrewAgentExecutor` to V2.
  warn(


In [2]:
import os
from utils import get_openai_api_key, pretty_print_result
from utils import get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

In [3]:
# CReating Agents 

In [4]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

## Creating Tools

### crewAI Tools

In [5]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [14]:
directory_read_tool = DirectoryReadTool(directory='./tools/instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

### Custom Tool
- Create a custom tool using crewAi's [BaseTool](https://docs.crewai.com/core-concepts/Tools/#subclassing-basetool) class

In [15]:
from crewai_tools import BaseTool

- Every Tool needs to have a `name` and a `description`.
- For simplicity and classroom purposes, `SentimentAnalysisTool` will return `positive` for every text.

In [16]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        #  custom code tool goes here
        return "positive"
sentiment_analysis_tool = SentimentAnalysisTool()

## Creating Tasks

- The Lead Profiling Task is using crewAI Tools.

In [17]:

lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

- The Personalized Outreach Task is using your custom Tool `SentimentAnalysisTool`, as well as crewAI's `SerperDevTool` (search_tool).

In [18]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

In [19]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose=2,
	memory=True
)

2025-04-20 01:55:30,743 - 4570183168 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


In [20]:
inputs = {
    "lead_name": "DataChain",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Chihab AKa",
    "position": "Crypto Trader",
    "milestone": "product AI Bot Trading launch"
}


result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of DataChain, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
I need to gather information on DataChain's company background, key personnel, recent milestones, and potential needs that align with our offerings. It's crucial to compile a detailed profile for a personalized engagement strategy.

Action: Search the internet
Action Input: {"search_query": "DataChain company profile"} 


Search results: Title: Datachain (Tokyo) - Valuation, Funding & Investors - PitchBook
Lin

In [21]:
from IPython.display import Markdown
Markdown(result)

Subject: Enhancing DataChain's Operations with CrewAI Solutions

Dear Chihab Aka,

I hope this message finds you well. I am reaching out to you on behalf of CrewAI, a leading provider of AI-driven solutions tailored to streamline operations and enhance performance for innovative companies like DataChain.

Having recently launched the AI Bot Trading product, DataChain has showcased a commitment to cutting-edge technology and forward-thinking strategies in the blockchain industry. At CrewAI, we admire your dedication to pushing boundaries and setting new standards in the field.

Based on the recent developments at DataChain, including the successful launch of "Project Pax" and collaborations with notable partners like Progmat and TOKI, it is evident that your team is focused on driving impactful change in the blockchain ecosystem.

I wanted to share some insights with you regarding how CrewAI's solutions have benefited similar companies in the past. Our AI agents have been instrumental in enhancing workflow management systems, optimizing memory resources, and improving data distribution infrastructure for various businesses operating in the blockchain space. By leveraging our capabilities, companies have reported increased efficiency, better performance, and enhanced collaboration among team members.

With a deep understanding of DataChain's business objectives and challenges, we believe that CrewAI's tailored engagement strategies can offer solutions that align perfectly with your needs. Whether it's streamlining collaboration processes, optimizing memory management within applications, or enhancing data distribution through AI-driven systems, our expertise can support DataChain in achieving its goals effectively.

I would love to schedule a call to discuss how CrewAI can specifically tailor our solutions to address DataChain's unique requirements and help propel your business forward. Please let me know a convenient time for you, and I will ensure to accommodate your schedule.

Looking forward to the opportunity to collaborate with you and support DataChain's continued success.

Warm regards,

[Your Name]
Lead Sales Representative
CrewAI